<a href="https://colab.research.google.com/github/AmitNikhade/AmitNikhade/blob/main/T5_finetuning_qa_closedbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # !pip install sentencepiece
# !pip uninstall transformers==4.19.2
!pip install transformers==4.18.0
# !pip uninstall sentencepiece -y

import transformers
# # import sentencepiece
transformers.__version__
# # sentencepiece.__version__
!pip install sentencepiece==0.1.96
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.0 MB 4.8 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 46.2 MB/s 
     |████████████████████████████████| 880 kB 62.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a29cf4b83f754e977ee28f763a3f5f96276683522a7ba8ba8dbf296e57ef7299
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.9 MB/s 
Looking in i

In [40]:
import torch
# import nlp
from transformers import AutoTokenizer
import transformers
# from transformers import Au
tokenizer = AutoTokenizer.from_pretrained('google/t5-small-ssm-nq')

In [56]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (example['question'], example['context'])
    # example['input_text'] =  example['question']+' '+ example['context']
    example['target_text'] = '%s </s>' % example['answers'][1]
    # example['target_text'] = example['answers'][1]
    return example

# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input_text'], pad_to_max_length=True, max_length=512)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target_text'], pad_to_max_length=True, max_length=16)

    encodings = {
        'input_ids': input_encodings['input_ids'], 
        'attention_mask': input_encodings['attention_mask'],
        'target_ids': target_encodings['input_ids'],
        'target_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [57]:
# tokenizer.encode_plus('question: %s  context: %s </s>' % ("how are you?", "I am fine")).input_ids

[822, 10, 149, 33, 25, 58, 2625, 10, 27, 183, 1399, 3, 1, 1]

In [58]:
# tokenizer.encode('question: %s  context: %s </s>' % ("how are you?", "I am fine"))

[822, 10, 149, 33, 25, 58, 2625, 10, 27, 183, 1399, 3, 1, 1]

In [59]:
import pandas as pd
df = pd.read_csv("/content/file1.csv")
# df
df = df.drop(columns=['answers'])
df = df.drop(columns=['Unnamed: 0'])
# # # df
df['answers'] = [[str(i),j] for i,j in zip(df['answer_start'],df['answer_text'])]
# df.columns

In [60]:
import datasets
from datasets import Dataset
# df = Dataset.from_pandas(df)

In [61]:
df = Dataset.from_pandas(df)
train_dataset1 = df.map(add_eos_to_examples)

train_dataset1 = train_dataset1.map(convert_to_features, batched=True)

  0%|          | 0/9660 [00:00<?, ?ex/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [62]:
columns = ['input_ids', 'target_ids', 'attention_mask', 'target_attention_mask']
train_dataset1.set_format(type='torch', columns=columns)

In [63]:
from torch.utils.data import Dataset, DataLoader
train_params = {
        "batch_size": 8,
        "shuffle": True,
        "num_workers": 0,
    }
training_loader = DataLoader(train_dataset1, **train_params)

In [64]:
import tqdm
def train(epoch, tokenizer, model, device, loader, optimizer):

    """
    Function to be called for training with the parameters passed from main function

    """

    model.train()
    for _, data in tqdm.tqdm_notebook(enumerate(loader, 0)):
        y = data["target_ids"].to(device, dtype=torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data["input_ids"].to(device, dtype=torch.long)
        mask = data["attention_mask"].to(device, dtype=torch.long)

        outputs = model(
            input_ids=ids,
            attention_mask=mask,
            decoder_input_ids=y_ids,
            labels=lm_labels,
            max_length=64,
    early_stopping=True,
    num_beams=10,
    num_return_sequences=3,
    no_repeat_ngram_size=2
        )
        loss = outputs[0]

        if _ % 10 == 0:
          print(str(epoch), str(_), str(loss))
        #     training_logger.add_row(str(epoch), str(_), str(loss))
        #     console.print(training_logger)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [65]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration
# model = T5ForConditionalGeneration.from_pretrained('t5-base')
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5-small-ssm-nq")
optimizer = torch.optim.Adam(
        params=model.parameters(), lr=0.001
    )

from torch import cuda

device = 'cuda' 
model = model.to(device)
  
model.train()
for epoch in range(5):
  train(epoch, tokenizer, model, device, training_loader, optimizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.


0it [00:00, ?it/s]

0 0 tensor(16.5242, device='cuda:0', grad_fn=<NllLossBackward0>)
0 10 tensor(3.6418, device='cuda:0', grad_fn=<NllLossBackward0>)
0 20 tensor(1.3847, device='cuda:0', grad_fn=<NllLossBackward0>)
0 30 tensor(1.1584, device='cuda:0', grad_fn=<NllLossBackward0>)
0 40 tensor(2.5430, device='cuda:0', grad_fn=<NllLossBackward0>)
0 50 tensor(1.5825, device='cuda:0', grad_fn=<NllLossBackward0>)
0 60 tensor(1.2375, device='cuda:0', grad_fn=<NllLossBackward0>)
0 70 tensor(1.1681, device='cuda:0', grad_fn=<NllLossBackward0>)
0 80 tensor(0.5528, device='cuda:0', grad_fn=<NllLossBackward0>)
0 90 tensor(0.7572, device='cuda:0', grad_fn=<NllLossBackward0>)
0 100 tensor(0.7562, device='cuda:0', grad_fn=<NllLossBackward0>)
0 110 tensor(1.5407, device='cuda:0', grad_fn=<NllLossBackward0>)
0 120 tensor(1.7941, device='cuda:0', grad_fn=<NllLossBackward0>)
0 130 tensor(1.3576, device='cuda:0', grad_fn=<NllLossBackward0>)
0 140 tensor(0.7563, device='cuda:0', grad_fn=<NllLossBackward0>)
0 150 tensor(1.1280,

0it [00:00, ?it/s]

1 0 tensor(0.4851, device='cuda:0', grad_fn=<NllLossBackward0>)
1 10 tensor(0.2406, device='cuda:0', grad_fn=<NllLossBackward0>)
1 20 tensor(0.8749, device='cuda:0', grad_fn=<NllLossBackward0>)
1 30 tensor(0.8524, device='cuda:0', grad_fn=<NllLossBackward0>)
1 40 tensor(0.5854, device='cuda:0', grad_fn=<NllLossBackward0>)
1 50 tensor(0.1009, device='cuda:0', grad_fn=<NllLossBackward0>)
1 60 tensor(0.3771, device='cuda:0', grad_fn=<NllLossBackward0>)
1 70 tensor(0.0658, device='cuda:0', grad_fn=<NllLossBackward0>)
1 80 tensor(0.0781, device='cuda:0', grad_fn=<NllLossBackward0>)
1 90 tensor(0.5068, device='cuda:0', grad_fn=<NllLossBackward0>)
1 100 tensor(0.5604, device='cuda:0', grad_fn=<NllLossBackward0>)
1 110 tensor(0.0090, device='cuda:0', grad_fn=<NllLossBackward0>)
1 120 tensor(0.4010, device='cuda:0', grad_fn=<NllLossBackward0>)
1 130 tensor(0.1798, device='cuda:0', grad_fn=<NllLossBackward0>)
1 140 tensor(0.2004, device='cuda:0', grad_fn=<NllLossBackward0>)
1 150 tensor(0.1544, 

In [93]:
# tokenizer

In [133]:
# model.save_pretrained('/content/drive/MyDrive/')
# tokenizer.save_pretrained('/content/drive/MyDrive/')
# model = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/')
# tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/')
# print(model)

In [13]:
# for i in training_loader:
#   # print(i['input_ids'].to(device, dtype = torch.long))
#   print([tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in i['input_ids'].to(device, dtype = torch.long)])
#   print([tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in i['target_ids'].to(device, dtype = torch.long)])
#   break

["question: What letter preceded the word lsie? context: But what appeal could it be? There were only four letters in the word which preceded 'Elsie,' and it ended in E ", 'question: Who is Elizabeth to marry in a fortnight? context: She is to be married in a fortnight to the Earl of Dovercourt ', 'question: Which of the imprudent letters was written to an impecunious young squire in the country? context: This fiend has several imprudent letters--imprudent, Watson, nothing worse--which were written to an impecunious young squire in the country ', 'question: How likely is it that the person will come back? context: " "You mean that the person will come again?" "It is very probable ', 'question: What was the last word in the book? context: "The last words in it were \'for God\'s sake,\'" he whispered, confidentially; "I am very anxious at getting no answer ', 'question: Who is the name of the man you can imagine? context: "You can imagine, Mr ', 'question: What did I do? context: " "How 

In [110]:
# que = tokenizer('what is your name?', return_tensors="pt").input_ids.to('cuda')
# ans = tokenizer("AMit Nikhade", return_tensors="pt").input_ids.to('cuda')
# model(input_ids=que, labels=ans)

In [41]:
# ids = torch.Tensor(g['attention_mask']).to(device, dtype = torch.long).reshape(1,len(g['attention_mask']))
# ids

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]], device='cuda:0')

In [89]:
device = 'cuda'
input_ids = tokenizer.encode_plus("What letter preceded the word lsie?", return_tensors="pt",pad_to_max_length=True, max_length=16).input_ids
gen_output = model.generate(input_ids.to(device, dtype = torch.long))[0]

print(tokenizer.decode(gen_output, skip_special_tokens=True))
# print(gen_output)

lsie 


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [42]:
ids = torch.Tensor(g['input_ids']).to(device, dtype = torch.long).reshape(1,len(g['input_ids']))
mask = torch.Tensor(g['attention_mask']).to(device, dtype = torch.long).reshape(1,len(g['attention_mask']))
# print(ids)
# print(mask)
generated_ids = model.generate(
              input_ids = ids
              )
# print(generated_ids)
# for i in generated_ids:
#   print(i)
print([tokenizer.decode(i, skip_special_tokens=True, clean_up_tokenization_spaces=True) for i in generated_ids])
#   break
# print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids])

['what letter befored the word lsie ']


In [14]:
def validate(epoch, tokenizer, model, device, loader):

  """
  Function to evaluate model for predictions

  """
  device = 'cuda'
  model.to(device)
  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for _, data in tqdm.tqdm_notebook(enumerate(loader, 0)):
          y = data['target_ids'].to(device, dtype = torch.long)
          ids = data['input_ids'].to(device, dtype = torch.long)
          mask = data['attention_mask'].to(device, dtype = torch.long)
          print(ids)
          # break
          generated_ids = model.generate(
              input_ids = ids,
              attention_mask = mask, 
              max_length=150, 
              num_beams=2,
              repetition_penalty=2.5, 
              length_penalty=1.0, 
              early_stopping=True
              )
          print(generated_ids)
          question = [tokenizer.decode(p, skip_special_tokens=True, clean_up_tokenization_spaces=True) for p in ids]
          preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
          target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
          # print(tokenizer.decode(data['input_ids'], skip_special_tokens=True, clean_up_tokenization_spaces=True))
          # if _%10==0:
          #     console.print(f'Completed {_}')
          # print("preds",preds)
          # print("actual:",actuals)
          print(question)
          predictions.extend(preds)
          actuals.extend(target)
          break
  # print(predictions, actuals)
  return predictions, actuals

In [29]:
validate(1, tokenizer, model, device, training_loader)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  del sys.path[0]


0it [00:00, ?it/s]

tensor([[ 822,   10,  363,  ...,    0,    0,    0],
        [ 822,   10,  366,  ...,    0,    0,    0],
        [ 822,   10,  363,  ...,    0,    0,    0],
        ...,
        [ 822,   10, 2645,  ...,    0,    0,    0],
        [ 822,   10, 2645,  ...,    0,    0,    0],
        [ 822,   10,  363,  ...,    0,    0,    0]], device='cuda:0')
tensor([[    0,     3,     9,  4816,  5011,     3,     1,     0],
        [    0,    73,  8715,     3,     1,     0,     0,     0],
        [    0,   572,    19,     3,    88,   270,     3,     1],
        [    0, 18763,     3,     1,     0,     0,     0,     0],
        [    0,     3,     9,  9062,    26,  3893,     3,     1],
        [    0,     3,     1,     0,     0,     0,     0,     0],
        [    0, 22960,     3,     1,     0,     0,     0,     0],
        [    0,     3,     1,     0,     0,     0,     0,     0]],
       device='cuda:0')
['question: What was the sound of when he laid down his burden? context: Turning his back upon us he lai

(['a sharp tap ',
  'unarmed ',
  'why is he here ',
  'Watson ',
  'a puzzled expression ',
  '',
  'Holmes ',
  ''],
 ['sharp tap ',
  'unarmed ',
  '"And why is he here?" ',
  'Watson ',
  'puzzled expression ',
  'Bob Carruthers ',
  'Holmes ',
  'collapsed '])